In [1]:
import tensorflow as tf
import pandas as pd


In [2]:
# constants
TEST_CSV_FILE_NAME = 'test_data.csv'
TRAIN_CSV_FILE_NAME = 'train_data.csv'
CSV_COLUMN_HEADERS = ['Value','Result']
FEATURE_COLUMN_HEADERS = list(filter(lambda item: item != 'Result', CSV_COLUMN_HEADERS))
CATEGORIES = ['odd', 'even']


In [3]:
# read in the file with pandas and create data frames.
train_x_df = pd.read_csv(TRAIN_CSV_FILE_NAME,header=0)
test_x_df = pd.read_csv(TEST_CSV_FILE_NAME, header=0)

# vocab = train_x_df['Type'].unique()

print(test_x_df.head())


   Value Result
0    -10   even
1     25    odd
2     43    odd
3    -46   even
4     71    odd


In [4]:
# Need separate container for labels
train_y_df = train_x_df.pop('Result')
test_y_df = test_x_df.pop('Result')

print(train_y_df)


0         even
1          odd
2         even
3          odd
4          odd
          ... 
810389     odd
810390    even
810391    even
810392    even
810393     odd
Name: Result, Length: 810394, dtype: object


In [5]:
# tf feature columns describing the inputs
# tf_feat_columns_array = [tf.feature_column.numeric_column(key='Value'), tf.feature_column.indicator_column(tf.feature_column.categorical_column_with_vocabulary_list(key='Type',vocabulary_list=vocab))]
tf_feat_columns_array = [tf.feature_column.numeric_column(key='Value')]
print(tf_feat_columns_array)

[NumericColumn(key='Value', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [6]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=tf_feat_columns_array, 
    hidden_units=[30,10], 
    n_classes=len(CATEGORIES),
    label_vocabulary=(CATEGORIES)
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\JEREMY~1\\AppData\\Local\\Temp\\tmpfj0yodbt', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [26]:
# Create input function

def test_input_func(features, labels, training:bool = True, batch_size:int = 50, num_epochs = 1):
    tf_dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    if (training):
        tf_dataset.shuffle(100)

    return tf_dataset.batch(batch_size).repeat(num_epochs)


In [27]:
classifier.train(lambda: test_input_func(train_x_df, train_y_df, training=True), steps=10)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\JEREMY~1\AppData\Local\Temp\tmpfj0yodbt\model.ckpt-543
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 543...
INFO:tensorflow:Saving checkpoints for 543 into C:\Users\JEREMY~1\AppData\Local\Temp\tmpfj0yodbt\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 543...
INFO:tensorflow:loss = 0.38040864, step = 543
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 553...
INFO:tensorflow:Saving checkpoints for 553 into C:\Users\JEREMY~1\AppData\Local\Temp\tmpfj0yodbt\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 553...
INFO:tensorflow:Loss for final step: 0.00014640886.


In [28]:
eval_result = classifier.evaluate(input_fn=lambda: test_input_func(test_x_df, test_y_df, training=False))


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2023-01-18T16:32:19
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\JEREMY~1\AppData\Local\Temp\tmpfj0yodbt\model.ckpt-553
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 13.49264s
INFO:tensorflow:Finished evaluation at 2023-01-18-16:32:32
INFO:tensorflow:Saving dict for global step 553: accuracy = 1.0, accuracy_baseline = 0.5011495, auc = 0.99999994, auc_precision_recall = 1.0, average_loss = 4.8007503e-05, global_step = 553, label/mean = 0.5011495, loss = 4.7983936e-05, precision = 1.0, prediction/mean = 0.50117284, recall = 1.0
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 553: C:\Users\JEREMY~1\AppData\Local\Temp\tmpfj0yodbt\model.ckpt-553


In [29]:
print(eval_result)

{'accuracy': 1.0, 'accuracy_baseline': 0.5011495, 'auc': 0.99999994, 'auc_precision_recall': 1.0, 'average_loss': 4.8007503e-05, 'label/mean': 0.5011495, 'loss': 4.7983936e-05, 'precision': 1.0, 'prediction/mean': 0.50117284, 'recall': 1.0, 'global_step': 553}


In [30]:
print(f"The accuracy of the neural network is {eval_result['accuracy'] * 100}%")

The accuracy of the neural network is 100.0%


In [35]:
number_predict = -23423

def predict_input_fn(features, batch_size=256):
    return tf.data.Dataset.from_tensor_slices((dict(features))).batch(batch_size)

FEATURE_COLUMN_HEADERS
predict = {}
predict['Value'] = [number_predict]
predict['Type'] = [str(type(number_predict))] # Tensorflow expects multiple predictions

predictions = classifier.predict(lambda: predict_input_fn(predict))

for pred_dict in predictions:
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%)'.format(
        CATEGORIES[class_id], 100 * probability))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\JEREMY~1\AppData\Local\Temp\tmpfj0yodbt\model.ckpt-553
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "even" (100.0%)
